In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install requests

In [ ]:
##################################################
# 1. 인증
##################################################

In [ ]:
# 1-1. 구글 API 인증
from google_auth_oauthlib.flow import InstalledAppFlow

# 구글 OAuth 클라이언트 ID Json 파일
gcreds_filename = 'auth/gcredentials.json'

# 캘린더에서 사용할 권한
SCOPES = ['https://www.googleapis.com/auth/calendar']

# 새로운 창에서 로그인 하시면 인증 정보를 얻게 됩니다.
flow = InstalledAppFlow.from_client_secrets_file(gcreds_filename, SCOPES)
gcreds = flow.run_local_server(port=0)

In [ ]:
# 1-2. 카카오톡 인증
# https://developers.kakao.com/docs/restapi/tool
# 해당 사이트에서 로그인 후 'Access token'을 얻어오세요
kcreds = {
    "access_token" : "<Access token>"
}
kheaders = {
    "Authorization": "Bearer " + kcreds.get('access_token')
}

In [ ]:
# 1-3. 네이버 인증
# https://developers.naver.com/apps
# 해당 사이트에서 로그인 후 "Cliend ID"와 "Client Secret"을 얻어오세요
ncreds = {
    "client_id": "<Client ID>",
    "client_secret" : "<Client Secret>"
}
nheaders = {
    "X-Naver-Client-Id" : ncreds.get('client_id'),
    "X-Naver-Client-Secret" : ncreds.get('client_secret')
}

In [ ]:
##################################################
# 2. 구글 캘린더 일정 가져오기
##################################################

In [ ]:
# 기본 라이브러리 import
import datetime
import requests
import urllib
import json

In [ ]:
# 구글 캘린더 API 서비스 객체 생성
from googleapiclient.discovery import build

service = build('calendar', 'v3', credentials=gcreds)

# 조회에 사용될 요청 변수 지정
calendar_id = 'primary'                   # 사용할 캘린더 ID
today = datetime.date.today().isoformat()
time_min = today + 'T00:00:00+09:00'      # 일정을 조회할 최소 날짜
time_max = today + 'T23:59:59+09:00'      # 일정을 조회할 최대 날짜
max_results = 1                           # 일정을 조회할 최대 개수
is_single_events = True                   # 반복 일정의 여부
orderby = 'startTime'                     # 일정 정렬

# 오늘 일정 가져오기
events_result = service.events().list(calendarId = calendar_id,
                                      timeMin = time_min,
                                      timeMax = time_max,
                                      maxResults = max_results,
                                      singleEvents = is_single_events,
                                      orderBy = orderby).execute()

events_result.get('items', [])

In [ ]:
##################################################
# 3. 일정 데이터 정제하기
##################################################
# 테스트를 위해 오늘 일정에서 한 개만 가져오겠습니다.
items = events_result.get('items')
item = items[0]

# 일정 제목
gsummary = item.get('summary')

# 일정 제목에서 [식사-국민대]에서 카테고리와 장소만 빼내옵니다.
gcategory, glocation = gsummary[gsummary.index('[')+1 : gsummary.index(']')].split('-')

# 해당 구글 캘린더 일정 연결 링크
gevent_url = item.get('htmlLink')

gaddress = item.get('location')
# 구글 주소를 정제합니다.
# ex) "국민대학교, 대한민국 서울특별시 성북구 정릉동 정릉로 77" => "대한민국 서울특별시 성북구 정릉동 정릉로 77"
if ',' in gaddress:
    gaddress = gaddress.split(',')[1].strip()

In [ ]:
##################################################
# 4. 일정 위치 주변 맛집 검색
##################################################

In [ ]:
# '식사' 카테고리만 처리하기 원한다면 if문 안으로 코드를 넣으세요.
# if '식사' in category:

# 인코딩된 맛집 검색어
enc_location = urllib.parse.quote(glocation + "맛집")

# 검색에 사용될 정보
# sort : "comment"을 넣으면 블로그 리뷰순으로 정렬
# query : 검색어
nparams = "sort=comment&query=" + enc_location

# 네이버 지역 검색 주소 및 검색할 정보
naver_local_url = "https://openapi.naver.com/v1/search/local.json?" + nparams

# 네이버에 일정 장소 주변 맛집을 검색합니다.
res = requests.get(naver_local_url, headers=nheaders)

# 검색 결과로 받아온 10개의 결과
places = res.json().get('items')

places[0]

In [ ]:
##################################################
# 5. 맛집 정보 카카오톡으로 보내기
##################################################

In [ ]:
# 5-1. 리스트 템플릿 형식 맞추기
# 카카오톡 리스트 템플릿은 최대 3개를 표현하기 때문에 3개만 가져오겠습니다.
contents = []
for place in places[:3]:
    ntitle = place.get('title')  # 장소 이름
    ncategory = place.get('category')  # 장소 카테고리
    ntelephone = place.get('telephone')  # 장소 전화번호
    nlocation = place.get('address')  # 장소 지번 주소

    # 각 장소를 클릭할 때 네이버 검색으로 연결해주기 위해 작성된 코드
    enc_location = urllib.parse.quote(nlocation + ' ' + ntitle)
    query = "query=" + enc_location

    # 장소 카테고리가 카페이면 카페 이미지
    # 이외에는 음식 이미지
    if '카페' in ncategory:
        image_url = "https://freesvg.org/img/pitr_Coffee_cup_icon.png"
    else:
        image_url = "https://freesvg.org/img/bentolunch.png?w=150&h=150&fit=fill"

    # 전화번호가 있다면 제목과 함께 넣어줍니다.
    if ntelephone:
        ntitle = ntitle + "\ntel) " + ntelephone

    # 카카오톡 리스트 템플릿 형식에 맞춰줍니다.
    content = {
        "title": "[" + ncategory + "] " + ntitle,
        "description": ' '.join(nlocation.split()[1:]),
        "image_url": image_url,
        "image_width": 50, "image_height": 50,
        "link": {
            "web_url": "https://search.naver.com/search.naver?" + query,
            "mobile_web_url": "https://search.naver.com/search.naver?" + query
        }
    }
    contents.append(content)

In [ ]:
# 일정 주소 네이버 연결 링크
enc_gaddress = urllib.parse.quote(gaddress)
query = "query=" + enc_gaddress
gaddr_url = "https://search.naver.com/search.naver?" + query

gaddr_url

In [ ]:
# 카카오톡 리스트 템플릿을 작성해봅니다.
ktemplate = {
    "object_type" : "list",
    "header_title" : "'%s' - 맛집 추천" % gsummary,
    "header_link" : {
        "web_url": gevent_url,
        "mobile_web_url" : gevent_url
    },
    "contents" : contents,
    "buttons" : [
        {
            "title" : "일정 자세히 보기",
            "link" : {
                "web_url": gevent_url,
                "mobile_web_url" : gevent_url
            }
        },
        {
            "title" : "일정 장소 보기",
            "link": {
                "web_url": gaddr_url ,
                "mobile_web_url": gaddr_url
            }
        }
    ],
}

# json 형식의 string을 보내야하기 때문에 json.dumps로 dict => string(json)으로 바꿔줍니다.
kpayload = {
    "template_object" : json.dumps(ktemplate)
}

kpayload

In [ ]:
# 5-2. 카카오톡 보내기

# 기본 템플릿(피드, 리스트, 위치, 커머스, 텍스트) : https://kapi.kakao.com/v2/api/talk/memo/default/send
# 스크랩 템플릿 : https://kapi.kakao.com/v2/api/talk/memo/scrap/send
# 커스텀 템플릿 : https://kapi.kakao.com/v2/api/talk/memo/send
kakaotalk_template_url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

res = requests.post(kakaotalk_template_url, data=kpayload, headers=kheaders)

res.json()
# {'result_code': 0}